In [11]:
from scipy.optimize import linprog
from math import sqrt
import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt 
import timeit
import pandas as pd
from scipy import stats
import time

random.seed(42)

In [12]:
# Init alpha, beta assumed to be positive
def init_alpha():
    alpha = [random.randint(-budget, 0) for i in range(num_treatments)]
    return alpha
def init_beta():
    beta = [random.randint(-budget, budget) for i in range(num_treatments)]
    return beta

In [13]:
# Price vector pi(i,t) = alpha(t) * pte(i,t) + beta(t). Dimensions num_subjects * num_treatments
def get_price_matrix(alpha, beta):
    price_matrix = [[(alpha[index] * pte_t + beta[index]) for index, pte_t in enumerate(pte)] for pte in pte_matrix]
    price_matrix = np.array(price_matrix)
    #print "get_price_matrix: Price matrix:", price_matrix
    return price_matrix

In [14]:
# Demand p*(i,t) matrix. Solve LP to get values. Dimensions num_subjects * num_treatments
def get_demand_matrix(price_matrix):
    prob_coefficient = [1] * num_treatments
    prob_sum = 1
    x0_bounds = (epsilon, 1-epsilon)
    x1_bounds = (epsilon, 1-epsilon)
    # dummy first row
    demand_matrix = np.zeros(num_treatments)
    for i in range(num_subjects):
        # Constraints:
        # 1. <p*(i), pi(i)> <= b(i) for every subject i
        # 2. sum of all p*(t) = prob_threshold for every subject i
        coefficients = price_matrix[i]
        thresholds = budget_matrix[i]
        result = linprog(c=-wtp_matrix[i], 
                         # A_ub needs to be 2D array  
                         A_ub=np.stack((coefficients, coefficients)),
                         b_ub=np.stack((thresholds, thresholds)),
                         # A_eq needs to be 2D array
                         A_eq=np.stack((prob_coefficient, prob_coefficient)),
                         b_eq=np.stack((prob_sum, prob_sum)),
                         # bounds = (0,1) for both demand prob.s
                         bounds = (x0_bounds, x1_bounds))
        demand_matrix = np.vstack((demand_matrix, result.x))
    # delete dummy first row
    demand_matrix = np.delete(demand_matrix, (0), axis=0)
    #print "get_demand_matrix: Demand matrix:", demand_matrix
    return demand_matrix

In [15]:
# Treatment_demand(t) = sum of demand(t) across all i. Dimensions 1 * num_treatments
def get_treatment_demand_matrix(demand_matrix):
    treatment_demand_matrix = np.zeros(num_treatments)
    for subject in range(num_subjects):
        for treatment in range(num_treatments):
            treatment_demand_matrix[treatment] += demand_matrix[subject, treatment]
    #print "get_treatment_demand_matrix: Treatment demand matrix:", treatment_demand_matrix
    return treatment_demand_matrix

In [16]:
# Excess_demand(t) = treatment_demand(t) - capacity(t). Dimensions 1 * num_treatments
def get_excess_demand_matrix(treatment_demand_matrix):
    excess_demand_matrix = treatment_demand_matrix - capacity_matrix
    #print "get_excess_demand_matrix: Excess demand matrix:", excess_demand_matrix
    return excess_demand_matrix

In [17]:
# Clearing error in market = sqrt(sum of excess_demand(t)^2 for every treatment t)
def get_clearing_error(excess_demand_matrix):
    # If demand is satisfied everywhere and total capacity > number of subjects, no clearing error
    if all(excess <= 0 for excess in excess_demand_matrix):
        print "get_clearing_error: Market clear, no clearing error!"
        return 0
    else:
        clearing_error = sqrt(sum([excess**2 for excess in excess_demand_matrix]))
        clearing_error = clearing_error / sum(capacity_matrix)
        print "get_clearing_error: Clearing error:", clearing_error
        return clearing_error

In [18]:
# Recalibrate alpha, beta values to set new prices
def get_alpha_new(alpha, excess_demand_matrix):
    alpha_new = alpha + excess_demand_matrix * alpha_scaling_factor
    for (i, a) in enumerate(alpha_new):
        if (a > 0):
            # alpha become +ve, so reset to random initialization
            alpha_new[i] = random.randint(-budget, 0)
    return alpha_new

def get_beta_new(beta, excess_demand_matrix):
    beta_new = beta + excess_demand_matrix * beta_scaling_factor
    return beta_new    

In [19]:
# Find market clearing price vector. The objective is to change alpha and beta values so that we reduce clearing error
def clear_market():
    
    # Initialize market prices and demand
    alpha = init_alpha()
    beta = init_beta()    
    price_matrix = get_price_matrix(alpha, beta)
    demand_matrix = get_demand_matrix(price_matrix)  
        
    excess_demand_matrix = get_excess_demand_matrix(get_treatment_demand_matrix(demand_matrix))
    clearing_error = get_clearing_error(excess_demand_matrix)
    
    # clearing_error_threshold = 0.01*(num_subjects)/(num_treatments)
    
    # clearing error is percentage of total capacity so we want the market to clear at 1%
    clearing_error_threshold = 0.01
    threshold_iterations = 10
    iterations = 0
    minimum_clearing_error = clearing_error
    alpha_star = 0
    beta_star = 0
    
    # Set new prices to clear market
    while True:
        if iterations > threshold_iterations:
            # new search start
            alpha = init_alpha()
            beta = init_beta()
            iterations = 0
            print "new search start"
#             print alpha, beta         
        else:
            # continue down current search
            alpha = get_alpha_new(alpha, excess_demand_matrix)
            beta = get_beta_new(beta, excess_demand_matrix)
        
        price_matrix = get_price_matrix(alpha, beta)
        demand_matrix = get_demand_matrix(price_matrix)
        excess_demand_matrix = get_excess_demand_matrix(get_treatment_demand_matrix(demand_matrix))
        clearing_error = get_clearing_error(excess_demand_matrix)
        
        # Store parameter values for minimum clearing error
        if clearing_error < minimum_clearing_error:
            minimum_clearing_error = clearing_error
            alpha_star = alpha.copy()
            beta_star = beta.copy()
        # cleared the market! 
        if minimum_clearing_error < clearing_error_threshold:
            break
        iterations += 1
    
    print "Minimum clearing error:", minimum_clearing_error
    print "Alpha_star:", alpha_star
    print "Beta star:", beta_star
    return (minimum_clearing_error, alpha_star, beta_star)

In [20]:
def simulate():
    while True: 
        min_error, alpha_star, beta_star = clear_market()
        price_star = get_price_matrix(alpha_star, beta_star)
        demand_star = get_demand_matrix(price_star)
        
        if min_error < 0.01: 
            break
    return demand_star

In [22]:
# hardcoded constants

num_subjects = 1540
num_treatments = 2
capacity_matrix = [663, 877]
budget = 100
budget_matrix = [budget] * num_subjects

# Scaling factor for alpha, beta to set new prices
alpha_scaling_factor = 0.5
beta_scaling_factor = budget/50

In [ ]:
# dict of form {dataset : demand_star} 
# every dataset is mapped to the market clearing probability distribution
demand_dict = {}

for d in range(1, 10):
    pte_df = pd.read_csv("data/PTE_"+str(d)+".csv")
    wtp_df = pd.read_csv("data/WTP_"+str(d)+".csv")
    
    pte_matrix = [[0, i] for i in pte_df['PTE'].values.tolist()]
    wtp_matrix = [[0, i] for i in wtp_df['WTP'].values.tolist()]
    
    # Convert lists to np.array type
    wtp_matrix = np.array(wtp_matrix)
    pte_matrix = np.array(pte_matrix)
    budget_matrix = np.array(budget_matrix)
    capacity_matrix = np.array(capacity_matrix)
    
    demand_i = simulate()
    demand_dict[d] = demand_i.tolist()
    
    print "finished dataset", d
    
df_results = pd.DataFrame.from_dict(demand_dict)

get_clearing_error: Clearing error: 0.39487781287
get_clearing_error: Clearing error: 0.143937223002
get_clearing_error: Clearing error: 0.110550978285
get_clearing_error: Clearing error: 0.0385141325652
get_clearing_error: Clearing error: 0.0232156755228
get_clearing_error: Clearing error: 0.0185362551749
get_clearing_error: Clearing error: 0.0133391516346
get_clearing_error: Clearing error: 0.010591596891
get_clearing_error: Clearing error: 0.00793331925918
Minimum clearing error: 0.00793331925918
Alpha_star: [-130.51215001 -180.48784999]
Beta star: [-222.04860002  246.04860002]
get_clearing_error: Clearing error: 0.220396918811
get_clearing_error: Clearing error: 0.136994245427


In [ ]:
df_results